In [1]:
import os
import numpy as np
import xarray as xr
import pandas as pd
from pathlib import Path
from sgkit_bgen import read_bgen
from bokeh.io import output_notebook
from dask.diagnostics import ResourceProfiler, ProgressBar
%load_ext autoreload
%autoreload 2
output_notebook()

Loading BokehJS ...

In [3]:
import sys
sys.path.insert(0,'../scripts')
from convert import BGENPaths, Contig, load_bgen, load_bgen_samples, load_bgen_dosage

In [4]:
paths = BGENPaths(
    bgen_path='/home/eczech/data/rs-ukb-local/bgen/ukb_imp_chrXY_v3.bgen',
    variants_path='/home/eczech/data/rs-ukb-local/bgen/ukb_mfi_chrXY_v3.txt',
    samples_path='/home/eczech/data/rs-ukb-local/bgen/ukb59384_imp_chrXY_v3_s486331.sample',
)
contig = Contig(index=25, name='XY')
ds = load_bgen(paths, contig)
ds

,Array,Chunk
Bytes,89.32 GB,75.97 MB
Shape,"(45906, 486443)","(3279, 5792)"
Count,1177 Tasks,1176 Chunks
Type,float32,numpy.ndarray
,Array,Chunk
Bytes,22.33 GB,18.99 MB
Shape,"(45906, 486443)","(3279, 5792)"
Count,2353 Tasks,1176 Chunks
Type,bool,numpy.ndarray


-----

Test actual computation

In [5]:
%%time
with ProgressBar():
    m = ds['call_dosage_mask'].mean(dim='samples').data.compute(scheduler='single-threaded')

[                                        ] | 0% Completed |  1min  7.2s
2020-08-18 13:35:47,697 | root | ERROR | Internal Python error in the inspect module.
Below is the traceback from this internal error.

Traceback (most recent call last):
  File "/home/eczech/miniconda3/envs/ukb-analysis/lib/python3.7/site-packages/IPython/core/magics/execution.py", line 1315, in time
    exec(code, glob, local_ns)
  File "<timed exec>", line 2, in <module>
  File "/home/eczech/miniconda3/envs/ukb-analysis/lib/python3.7/site-packages/dask/base.py", line 167, in compute
    (result,) = compute(self, traverse=False, **kwargs)
  File "/home/eczech/miniconda3/envs/ukb-analysis/lib/python3.7/site-packages/dask/base.py", line 447, in compute
    results = schedule(dsk, keys, **kwargs)
  File "/home/eczech/miniconda3/envs/ukb-analysis/lib/python3.7/site-packages/dask/local.py", line 527, in get_sync
    return get_async(apply_sync, 1, dsk, keys, **kwargs)
  File "/home/eczech/miniconda3/envs/ukb-analysis/

TypeError: object of type 'NoneType' has no len()

----

PyBGEN debug

In [13]:
from pybgen import PyBGEN
bgen = PyBGEN(paths.bgen_path)

In [14]:
info, dosage = next(bgen)

In [38]:
info

<Variant rs370048753 chrPAR1:60014_T/C>

In [16]:
dosage.shape

(486443,)

In [39]:
bgen.close()